In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [6]:
# Inicialize o driver do Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL da página a ser scrapeada
url = 'https://www.in.gov.br/en/web/dou/-/portaria-mesp-n-59-de-21-de-maio-de-2024-561242616'
driver.get(url)

# Scroll down até o final da página
body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(10):  # Ajuste o número de scrolls conforme necessário
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

# Parse o HTML usando BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Encontre a tabela com a classe "dou-table"
table = soup.find('table', class_='dou-table')

# Inicialize as listas para armazenar os dados
nomes = []
cpfs = []
esportes = []

# Verifique se a tabela foi encontrada
if table:
    tbody = table.find('tbody')
    trs = tbody.find_all('tr')
    
    for tr in trs:
        tds = tr.find_all('td')
        
        # Verifique se há elementos <td> suficientes no <tr>
        if len(tds) >= 4:
            nome_p = tds[1].find('p', class_='dou-paragraph')
            cpf_p = tds[2].find('p', class_='dou-paragraph')
            esporte_p = tds[3].find('p', class_='dou-paragraph')
            
            # Verifique se os elementos <p> foram encontrados e extraia o texto
            if nome_p and cpf_p and esporte_p:
                nomes.append(nome_p.text.strip())
                cpfs.append(cpf_p.text.strip())
                esportes.append(esporte_p.text.strip())

# Imprima as listas para verificar os dados extraídos
print("Nomes:", nomes)
print("CPFs:", cpfs)
print("Esportes:", esportes)

# Feche o driver
driver.quit()

Nomes: ['Nome', 'AGATHA BEDNARCZUK RIPPEL', 'AILTON DE ANDRADE BENTO DE SOUZA', 'ALEX DOUGLAS PIRES DA SILVA', 'ALEXSANDRO DO NASCIMENTO DE MELO', 'ALINE BIEGER', 'ALINE DA SILVA RODRIGUES', 'ALMIR CUNHA DOS SANTOS', 'ALTOBELI SANTOS DA SILVA', 'AMANDA FERREIRA SANCHES', 'ANA CAROLINA DE JESUS AZEVEDO', 'ANA CAROLINA VIEIRA', 'ANA CLAUDIA DOS SANTOS CALDEIRA', 'ANA CLAUDIA LEMOS SILVA', 'ANA CLAUDIA MARIA DA SILVA', 'ANA PAULA MADRUGA DE SOUZA', 'ANA RAQUEL MONTENEGRO BATISTA LINS', 'ANDERSON FREITAS HENRIQUES', 'ANDRE ARTHUR DUTRA', 'ANDRESSA OLIVEIRA DE MORAIS', 'ANDREY MUNIZ DE CASTRO', 'ANDREY PEREIRA GARBE', 'ANE MARCELLE GOMES DOS SANTOS', 'ANTONIO TENORIO DA SILVA', 'AUGUSTO DUTRA DA SILVA DE OLIVEIRA', 'BEATRIZ BORGES CARNEIRO', 'BEATRIZ PIMENTEL DIZOTTI', 'BRENO MARTINS CORREIA', 'BRUNA JESSICA OLIVEIRA FARIAS', 'BRUNA YUMI TAKAHASHI', 'BRUNO BECKER DA SILVA', 'BRUNO PINHEIRO CARRA', 'CAIO AMORIM MUNIZ DE OLIVEIRA', 'CAIO RODRIGUES PUMPUTIS', 'CARLOS ALBERTO CARBINATTI JUNIOR'

In [7]:
len(nomes)

211

In [3]:
data = {
    'Nome': nomes,
    'CPFs': cpfs,
    'Esporte': esportes
}

df = pd.DataFrame(data)
df.to_csv('Atletas Portaria MESP.csv', index=False)